## Imports

In [1]:
# This command ensures that if you edit py files, the changes are instantly loaded here without restarting.
%load_ext autoreload
%autoreload 2
    
from data_processing import run_data_pipeline
from fantasy_model import FantasyModel
from prediction import predict_upcoming_week

## Configuration

In [2]:
CURRENT_SEASON = 2025
NEXT_WEEK = 15

## Fetch and Update Data

In [3]:
# Fetch 2015-2024 to get full history
years = list(range(2015, 2026))
run_data_pipeline(years_to_fetch=years, positions=['QB', 'RB', 'WR', 'TE'], output_dir='data')

Fetching weekly data for seasons: [2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023, 2024, 2025]...
  Successfully fetched data for 2015.
  Successfully fetched data for 2016.
  Successfully fetched data for 2017.
  Successfully fetched data for 2018.
  Successfully fetched data for 2019.
  Successfully fetched data for 2020.
  Successfully fetched data for 2021.
  Successfully fetched data for 2022.
  Successfully fetched data for 2023.
  Successfully fetched data for 2024.
  Successfully fetched data for 2025.
All data has been combined.
Parsing data...
DataFrame has been parsed. Kept 40 columns.
Filled all missing numeric values with 0.
Saving data to data/nfl_weekly_data.csv...
Data successfully saved!


## Initialize and Train Model

In [20]:
# We point it to the data
model = FantasyModel(data_filepath='data/nfl_weekly_data.csv')

# Run the full training pipeline
model.run_pipeline(do_eda=False, test_season=2025, burn_in_years=1)

Starting data pipeline...
Successfully loaded data from data/nfl_weekly_data.csv
Shape of raw data: (63025, 40)
Engineering features...
--- Engineering Features ---
Filtered data to ['QB', 'RB', 'WR', 'TE'] only. Shape: (63025, 40)
Calculating 'Recent Form' (EWMA) and 'Season-to-Date' (Expanding) features...
Finished 'Recent Form' and 'Season-to-Date' features.
Calculating Dual Depth Chart Ranks (Stable vs. Recent)...
Calculating 'Defense vs Position' stats...
Finished Defensive Matchup features.
Calculating 'Team Offense' stats...
Finished Team Offense features.
Finished 'Career Baseline' features.
--- Feature Engineering Complete ---
Preprocessing data...
--- Preprocessing Data ---
Filled all missing numeric values with 0.
Successfully one-hot encoded categorical features.
Dropped raw string columns: ['opponent_team', 'recent_team']


,player_id,player_display_name,season,week,fantasy_points_ppr,ewm_completions,expanding_completions,ewm_attempts,expanding_attempts,ewm_passing_yards,...,last_season_receiving_air_yards,last_season_receiving_yards_after_catch,last_season_target_share,last_season_air_yards_share,last_season_offense_snaps,last_season_fantasy_points_ppr,position_QB,position_RB,position_TE,position_WR
0,00-0007091,Matt Hasselbeck,2015,4,15.38,0.000000,0.000000,0.000000,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,1,0,0,0
1,00-0007091,Matt Hasselbeck,2015,5,16.22,30.000000,30.000000,47.000000,47.0,282.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,1,0,0,0
2,00-0007091,Matt Hasselbeck,2015,11,13.32,26.000000,24.000000,41.000000,38.0,259.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,1,0,0,0
3,00-0007091,Matt Hasselbeck,2015,12,20.30,25.000000,23.666667,38.000000,36.0,243.666667,...,0.0,0.0,0.0,0.0,0.0,0.0,1,0,0,0
4,00-0007091,Matt Hasselbeck,2015,13,4.86,25.333333,24.250000,39.333333,37.5,267.444444,...,0.0,0.0,0.0,0.0,0.0,0.0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63020,00-0040784,Quinshon Judkins,2025,10,10.50,0.000000,0.000000,0.000000,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0,1,0,0
63021,00-0040784,Quinshon Judkins,2025,11,5.90,0.000000,0.000000,0.000000,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0,1,0,0
63022,00-0040784,Quinshon Judkins,2025,12,16.70,0.000000,0.000000,0.000000,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0,1,0,0
63023,00-0040784,Quinshon Judkins,2025,13,15.90,0.000000,0.000000,0.000000,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0,1,0,0


Saved model-ready data for inspection to: data/final_model_ready_data.csv
Splitting data...
--- Splitting Data (Test: 2025, Burn-in: 1 yrs) ---
Raw Data Start: 2015
Training Start: 2016 (First 1 years used for history only)
Training set shape: (52748, 108)
Testing set shape: (4634, 108)
Data successfully split.
Training model...
--- Training Model ---
Model initialized. Starting training...
Model saved to xgb_model.json
--- Model Training Complete ---
Model stopped at iteration: 412
Evaluating model...
--- Evaluating Model ---
--- Model Evaluation Results ---
  Mean Absolute Error (MAE):  4.3330 (fantasy points)
  Root Mean Squared Error (RMSE): 5.9389 (fantasy points)
  Custom Accuracy Metrics:
  Natural Noise of Test Set (StdDev): 7.90 pts
  Accuracy (within Natural Noise): 84.94%
  Accuracy (within 5.0 pts):      69.87%
----------------------------------
Explaining model...
--- Explaining Model (Calculating SHAP values) ---
--- SHAP Value Calculation Complete ---
Plotting feature im

## Load Saved Model

In [5]:
# Initialize the Class
model = FantasyModel(data_filepath='data/nfl_weekly_data.csv')

# Load the Data & model
model.load_data()
model.load_saved_model("xgb_model.json")

Successfully loaded data from data/nfl_weekly_data.csv
Shape of raw data: (63025, 40)
--- Loading Saved Model from xgb_model.json ---
Model successfully loaded.


## Predict Week

In [6]:
projections = predict_upcoming_week(model=model, season=CURRENT_SEASON, week=NEXT_WEEK)

# Display Top 20
print(projections.head(20).reset_index(drop=True))

# Save to CSV
projections.to_csv(f"predictions/{CURRENT_SEASON}_week_{NEXT_WEEK}_projections.csv", index=False)
print(f"Saved projections to predictions/{CURRENT_SEASON}_week_{NEXT_WEEK}_projections.csv")

--- Predicting 2025 Week 15 ---
Forcing a clean data reload to purge ghost rows...
Successfully loaded data from data/nfl_weekly_data.csv
Shape of raw data: (63025, 40)
Purging any existing data for 2025 Week 15...
--- Engineering Features ---
Filtered data to ['QB', 'RB', 'WR', 'TE'] only. Shape: (63993, 40)
Calculating 'Recent Form' (EWMA) and 'Season-to-Date' (Expanding) features...
Finished 'Recent Form' and 'Season-to-Date' features.
Calculating 'Defense vs Position' stats...
Finished Defensive Matchup features.
Calculating 'Team Offense' stats...
Finished Team Offense features.
Finished 'Career Baseline' features.
--- Feature Engineering Complete ---
--- Preprocessing Data ---
Filled all missing numeric values with 0.
Successfully one-hot encoded categorical features.
Dropped raw string columns: ['opponent_team', 'recent_team']
    player_display_name position recent_team opponent_team  projected_points
0            Josh Allen       QB         BUF            NE         20.835546
